In [144]:
import pandas as pd
from IPython.display import display
from PIL import Image
df_train_ = pd.read_csv('../../datasets/covid/escala_grises/train.csv')
df_test_ = pd.read_csv('../../datasets/covid/escala_grises/test.csv')
df_train_['class_num'] = df_train_['class'].apply(lambda x: 1 if x == 'positive' else 0)
df_test_['class_num'] = df_test_['class'].apply(lambda x: 1 if x == 'positive' else 0)
df_train_.head()

,class,patient_id,filename,class_num
0,negative,379,1e64990d1b40c1758a2aaa9c7f7a85_jumbo.jpg,0
1,negative,379,7223b8ad031187d9a142d7f7ca02c9_jumbo.jpg,0
2,negative,380,3392dc7d262e28423caca517f98c2e_jumbo.jpg,0
3,negative,380,ec3a480c0926ded74429df416cfb05_jumbo.jpg,0
4,negative,382,a72aeb349a63c79ed24e473c434efe_jumbo.jpg,0


In [145]:
df_train_['class'].value_counts()

class
positive    57199
negative    10664
Name: count, dtype: int64

In [146]:
def splitDataFrames(df_train,df_test,n_train,n_test,random_state=42,proporcion_positiva=0.1,label_col='class_num'):
    # Dividir los dataframes en train y test
    n_positivos = int(n_train*proporcion_positiva)
    n_negativos = n_train-n_positivos
    df_train_positivo = df_train[df_train[label_col]==1]
    df_train_negativo = df_train[df_train[label_col]==0]
    df_test_positivo = df_test[df_test[label_col]==1]
    df_test_negativo = df_test[df_test[label_col]==0]
    # Dividir los dataframes en train y test
    df_train_positivo = df_train_positivo.sample(n=n_positivos,random_state=random_state)
    df_train_negativo = df_train_negativo.sample(n=n_negativos,random_state=random_state)
    n_positivos = int(n_test*proporcion_positiva)
    n_negativos = n_test-n_positivos
    df_test_positivo = df_test_positivo.sample(n=n_positivos,random_state=random_state)
    df_test_negativo = df_test_negativo.sample(n=n_negativos,random_state=random_state)
    # Concatenar los dataframes
    df_train = pd.concat([df_train_positivo,df_train_negativo],ignore_index=True)
    df_test = pd.concat([df_test_positivo,df_test_negativo],ignore_index=True)
    return df_train,df_test

In [147]:
# 400 Imagenes de entrenamiento con 50% de positivos y 50% de negativos
# 100 Imagenes de test con 50% de positivos y 50% de negativos
df_train, _ = splitDataFrames(df_train_,df_test_,n_train=500,n_test=1,random_state=0,proporcion_positiva=0.5,label_col='class_num')
print(df_train['class'].value_counts())

class
positive    250
negative    250
Name: count, dtype: int64


# Distribución de Images

|Persona|Cantidad|
|-|-|
|200|Imanol|
|200|Magno|
|100|Erick|

In [148]:
import os
import shutil
def copiarImagenes(imagenes, folder_origen, folder_destino):
    rutas_destino = []
    for imagen in imagenes:
        ruta_origen = os.path.join(folder_origen, imagen)
        ruta_destino = os.path.join(folder_destino, imagen)
        shutil.copy(ruta_origen, ruta_destino)
        rutas_destino.append(ruta_destino)
    return rutas_destino

In [149]:
df_magno = df_train.iloc[:200]
df_imanol = df_train.iloc[200:400]
df_erick = df_train.iloc[400:500]
imganes_magno = set(df_magno['filename'])
imganes_imanol = set(df_imanol['filename'])
imagenes_erick = set(df_erick['filename'])
imagenes_set = set(df_train['filename'])
if len(imganes_magno.intersection(imganes_imanol)) != 0:
    raise Exception('Hay imagenes en comun entre los dos dataframes')
if len(imganes_magno.intersection(imagenes_set)) != 200:
    raise Exception('No se incluyeron las 200 imagenes de magno')
if len(imganes_imanol.intersection(imagenes_set)) != 200:
    raise Exception('No se incluyeron las 200 imagenes de imanol')
r = []
r+= copiarImagenes(list(imganes_magno)[:],'../../datasets/covid/escala_grises/train','../../datasets/covid/escala_grises/proceso_etiquetas/etiquetas/magno/img')
r+= copiarImagenes(list(imganes_imanol)[:],'../../datasets/covid/escala_grises/train','../../datasets/covid/escala_grises/proceso_etiquetas/etiquetas/imanol/img')
r+= copiarImagenes(list(imagenes_erick)[:],'../../datasets/covid/escala_grises/train','../../datasets/covid/escala_grises/proceso_etiquetas/etiquetas/erick/img')
df_train['folder'] = ['magno/img']*200+['imanol/img']*200+['erick/img']*100
df_train.head()

,class,patient_id,filename,class_num,folder
0,positive,S314259,sub-S314259_ses-E30604_run-1_bp-chest_vp-ap_dx...,1,magno/img
1,positive,S319789,sub-S319789_ses-E40738_run-1_bp-chest_vp-pa_dx...,1,magno/img
2,positive,S309894,sub-S309894_ses-E37950_run-1_bp-chest_vp-ap_dx...,1,magno/img
3,positive,A236763,A236763-02-23-1901-NA-CHEST_AP_PORT-32482-3.00...,1,magno/img
4,positive,S322790,sub-S322790_ses-E46033_run-1_bp-chest_vp-ap_cr...,1,magno/img


In [150]:
df_train.to_csv('../../datasets/covid/escala_grises/proceso_etiquetas/etiquetas/etiquetas.csv',index=False)